In [89]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# import RDKit

np.random.seed(123)

%cd /Users/cgu3/Documents/conditional-graph-variational-autoencoder
%pwd

/Users/cgu3/Documents/conditional-graph-variational-autoencoder


'/Users/cgu3/Documents/conditional-graph-variational-autoencoder'

In [90]:
# %qtconsole

In [91]:
dat = pd.read_csv('data/interactions/metabolites-detected-and-quantified.csv')
dat.head()

,HMDB_ID,NAME,SMILES,INCHIKEY,CHEMICAL_FORMULA,AVERAGE_MASS,MONO_MASS
0,HMDB0000001,1-Methylhistidine,CN1C=NC(C[C@H](N)C(O)=O)=C1,BRMWTNUJHUMWMS-LURJTMIESA-N,C7H11N3O2,169.1811,169.085127
1,HMDB0000002,"1,3-Diaminopropane",NCCCN,XFNJVJPLKCPIBV-UHFFFAOYSA-N,C3H10N2,74.1249,74.084398
2,HMDB0000005,2-Ketobutyric acid,CCC(=O)C(O)=O,TYEYBOSBBBHJIV-UHFFFAOYSA-N,C4H6O3,102.0886,102.031694
3,HMDB0000008,2-Hydroxybutyric acid,[H]OC(=O)[C@@]([H])(O[H])C([H])([H])C([H])([H]...,AFENDNXGAFYKQO-VKHMYHEASA-N,C4H8O3,104.1050,104.047344
4,HMDB0000010,2-Methoxyestrone,[H][C@@]12CCC(=O)[C@@]1(C)CC[C@]1([H])C3=C(CC[...,WHEUWNKSCXYKBU-QPWUGHHJSA-N,C19H24O3,300.3921,300.172545


In [92]:
# check whether there is None or ''  in the column 'SMILES'
dat['SMILES'].isnull().sum()

0

In [93]:
from rdkit import Chem

In [94]:
from rdkit.Chem.MolStandardize import rdMolStandardize
# convert dat['canonicalsmiles'] into a list and then using Chem.MolFromSmiles
mols = [Chem.MolFromSmiles(x) for x in dat['SMILES']]
# return index of 'none' in the list
none_index = [i for i, x in enumerate(mols) if x is None]
none_index
# check the SMILES of the 'none' index
dat.iloc[none_index]

[]

,HMDB_ID,NAME,SMILES,INCHIKEY,CHEMICAL_FORMULA,AVERAGE_MASS,MONO_MASS


In [95]:
from rdkit import DataStructs
from rdkit.Chem import AllChem
fpgen = AllChem.GetRDKitFPGenerator(fpSize=1024)
fps = [fpgen.GetFingerprint(x) for x in mols]

In [96]:
# get ECFP4 fingerprint
fpgen = AllChem.GetMorganGenerator(radius=2, fpSize=1024)
fps = [fpgen.GetFingerprint(x) for x in mols]

In [97]:
# to np array
fps_np = np.array([np.array(list(x.ToBitString())).astype(int) for x in fps])

In [76]:
# np.array into torch.tensor
fps_torch = torch.tensor(fps_np)

In [77]:
fps_torch.shape

torch.Size([3382, 1024])

In [78]:
# save np.array to file
np.save('data/metabolites/ECFP.npy', fps_torch)

In [80]:
import pickle